In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
import pickle
import pipreqs
from sklearn.preprocessing import LabelEncoder

In [2]:
dataset = pd.read_excel('PPG-BP dataset.xlsx') 

In [3]:
dataset.head(3)

,Num.,subject_ID,Sex_M_F,Age_year,Height_cm,Weight_kg,Systolic Blood Pressure_mmHg,Diastolic Blood Pressure_mmHg,Heart_Rate_b_m,BMI_kg_m2,Hypertension,Diabetes,cerebral_infarction,cerebrovascular_disease
0,1,2,Female,45,152,63,161,89,97,27.268006,Stage 2 hypertension,NaN,NaN,NaN
1,2,3,Female,50,157,50,160,93,76,20.284799,Stage 2 hypertension,NaN,NaN,NaN
2,3,6,Female,47,150,47,101,71,79,20.888889,Normal,NaN,NaN,NaN


In [4]:
dataset['Diabetes'].value_counts()

Type 2 Diabetes    37
Diabetes            1
Name: Diabetes, dtype: int64

In [5]:
dataset.loc[dataset['Diabetes'] == "Type 2 Diabetes", 'Diabetes'] = 1
dataset.loc[dataset['Diabetes'] == "Diabetes", 'Diabetes'] = 1

dataset.loc[dataset['Sex_M_F'] == "Male", 'Sex_M_F'] = 1
dataset.loc[dataset['Sex_M_F'] == "Female", 'Sex_M_F'] = 2

In [6]:
dataset['Diabetes'].value_counts()

1    38
Name: Diabetes, dtype: int64

In [7]:
# dataset = dataset.fillna(0)
dataset = dataset.drop(['Num.', 'subject_ID'], axis=1)

In [8]:
dataset.head(100)

,Sex_M_F,Age_year,Height_cm,Weight_kg,Systolic Blood Pressure_mmHg,Diastolic Blood Pressure_mmHg,Heart_Rate_b_m,BMI_kg_m2,Hypertension,Diabetes,cerebral_infarction,cerebrovascular_disease
0,2,45,152,63,161,89,97,27.268006,Stage 2 hypertension,NaN,NaN,NaN
1,2,50,157,50,160,93,76,20.284799,Stage 2 hypertension,NaN,NaN,NaN
2,2,47,150,47,101,71,79,20.888889,Normal,NaN,NaN,NaN
3,1,45,172,65,136,93,87,21.971336,Prehypertension,NaN,NaN,NaN
4,2,46,155,65,123,73,73,27.055151,Prehypertension,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
95,2,80,156,60,123,56,58,24.654832,Prehypertension,NaN,cerebral infarction,NaN
96,2,79,148,40,128,64,73,18.261505,Prehypertension,NaN,NaN,insufficiency of cerebral blood supply
97,2,85,155,60,149,71,67,24.973985,Stage 1 hypertension,NaN,NaN,cerebrovascular disease
98,2,69,150,61,141,67,61,27.111111,Stage 1 hypertension,NaN,cerebral infarction,NaN


In [10]:
dataset['cerebrovascular_disease'].value_counts()

insufficiency of cerebral blood supply    15
cerebrovascular disease                   10
Name: cerebrovascular_disease, dtype: int64

In [11]:
le = LabelEncoder()

dataset['Hypertension']            = le.fit_transform(dataset['Hypertension'])
dataset['cerebral_infarction']     = le.fit_transform(dataset['cerebral_infarction'])
dataset['cerebrovascular_disease'] = le.fit_transform(dataset['cerebrovascular_disease'])

# dataset = dataset.fillna(0)
dataset.head(100)

,Sex_M_F,Age_year,Height_cm,Weight_kg,Systolic Blood Pressure_mmHg,Diastolic Blood Pressure_mmHg,Heart_Rate_b_m,BMI_kg_m2,Hypertension,Diabetes,cerebral_infarction,cerebrovascular_disease
0,2,45,152,63,161,89,97,27.268006,3,NaN,1,2
1,2,50,157,50,160,93,76,20.284799,3,NaN,1,2
2,2,47,150,47,101,71,79,20.888889,0,NaN,1,2
3,1,45,172,65,136,93,87,21.971336,1,NaN,1,2
4,2,46,155,65,123,73,73,27.055151,1,NaN,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...
95,2,80,156,60,123,56,58,24.654832,1,NaN,0,2
96,2,79,148,40,128,64,73,18.261505,1,NaN,1,1
97,2,85,155,60,149,71,67,24.973985,2,NaN,1,0
98,2,69,150,61,141,67,61,27.111111,2,NaN,0,2


In [12]:
dataset = dataset.fillna(0)

In [13]:
dataset['Diabetes'].value_counts()

0    181
1     38
Name: Diabetes, dtype: int64

In [14]:
diabetes_dataset.groupby('Diabetes').mean()

NameError: name 'diabetes_dataset' is not defined

In [15]:
dataset.groupby('Diabetes').mean()

,Sex_M_F,Age_year,Height_cm,Weight_kg,Systolic Blood Pressure_mmHg,Diastolic Blood Pressure_mmHg,Heart_Rate_b_m,BMI_kg_m2,Hypertension,cerebral_infarction,cerebrovascular_disease
Diabetes,,,,,,,,,,,
0,1.524862,56.751381,161.292818,59.624309,127.281768,71.519337,73.756906,22.869509,0.950276,0.889503,1.80663
1,1.526316,59.157895,160.921053,62.894737,131.105263,73.421053,73.078947,24.240552,1.078947,1.000000,2.00000


In [16]:
X = dataset.drop(columns = 'Diabetes', axis=1)
Y = dataset['Diabetes']

In [17]:
scaler = StandardScaler()

In [18]:
scaler.fit(X)

StandardScaler()

In [19]:
standardized_data = scaler.transform(X)

In [20]:
X = standardized_data
Y = dataset['Diabetes']

In [21]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, stratify=Y, random_state=2)

In [22]:
classifier = svm.SVC(kernel='linear')

In [23]:
classifier.fit(X_train, Y_train)

SVC(kernel='linear')

In [24]:
X_train_prediction = classifier.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [25]:
X_test_prediction = classifier.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [26]:
input_data = (2, 45, 152, 63, 161, 90, 96, 25, 3, 1, 2)

# changing the input_data to numpy array
input_data_as_numpy_array = np.asarray(input_data)

# reshape the array as we are predicting for one instance
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)

# standardize the input data
std_data = scaler.transform(input_data_reshaped)
print(std_data)

prediction = classifier.predict(std_data)
print(prediction)

if (prediction[0] == 0):
  print('The person is not diabetic')
else:
  print('The person is diabetic')

[[ 0.95097204 -0.76833671 -1.12758724  0.23679803  1.62581614  1.63729046
   2.08699338  0.47370416  2.15498436  0.31702131  0.33647625]]
[0]
The person is not diabetic


/Users/alexandra/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [27]:
import pickle

In [28]:
filename = "trained_model_ppg.pkl"
pickle.dump(classifier, open(filename, 'wb'))

In [29]:
#loading save model
loaded_model = pickle.load(open('trained_model_ppg.pkl', 'rb'))

In [30]:
input_data = (2, 45, 152, 63, 161, 90, 96, 25, 3, 1, 2)

# changing the input_data to numpy array
input_data_as_numpy_array = np.asarray(input_data)

# reshape the array as we are predicting for one instance
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)

# standardize the input data
std_data = scaler.transform(input_data_reshaped)
print(std_data)

prediction = classifier.predict(std_data)
print(prediction)

if (prediction[0] == 0):
  print('The person is not diabetic')
else:
  print('The person is diabetic')

[[ 0.95097204 -0.76833671 -1.12758724  0.23679803  1.62581614  1.63729046
   2.08699338  0.47370416  2.15498436  0.31702131  0.33647625]]
[0]
The person is not diabetic


/Users/alexandra/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
